In [1]:
# standard lib
#import sys
from pathlib import Path
import os
import sys
import shutil
# scientific
import numpy as np
import pandas as pd
import h5py
import random

# plotting
%matplotlib inline
#%matplotlib notebook
from matplotlib.pyplot import *
import matplotlib as mpl

# local
#sys.path.append(...)

In [3]:
# global parameters
split_dir = Path('/media/hdd/clguo/ellak/sk06_res/splits')
# prediction
root_dir = Path('/media/hdd/clguo/ellak/sk06_res/com/layer')
feature_root = Path('/media/hdd/clguo/ellak/sk06_res/com_inp')
features = ['whole', 'colasym', 'center', 'border', 'bwv']
feature_names = dict(whole='whole', colasym='color asymmetry', center='center', border='border', bwv='blue white veil traits')
threshold = 0.00116

In [3]:
# prediction support
splits = {fold_: pd.read_csv(split_dir / f'splitA{fold_}.txt', header=None, delim_whitespace=True, names=['split', 'cat', 'name'])
          for fold_ in range(5)}
#print (len(splits[0]))
def get_fold_idx(name):
    # find fold in which name is in test split
    for fold in range(5):
        split = splits[fold]
        test_rows = split.loc[split.split == 'test'].copy()
        rows = test_rows.loc[test_rows.name == name]
        if len(rows) != 1:
            continue
        test_rows.sort_values(by=['cat', 'name'], inplace=True)
        test_rows.reset_index(inplace=True)
        idx = test_rows.index[test_rows.name == name][0]
        
        return fold, idx
    raise KeyError(f'"{name}" is in neither test set')
    
def get_true_pred(name):
    # calculated from averaged y_pred for given fold runs
    fold, idx = get_fold_idx(name)
    y_pred_all = []
    y_true_saved = None
    for dir in root_dir.glob(f'f{fold}_r*'): #doubt
        path = dir / 'test.h5'
        #print(path)
        with h5py.File(str(path), 'r') as h5f:
            y_pred = h5f['pred'][()]
            y_true = h5f['true'][()]
      
        assert y_pred.shape[1] == 1
        if y_true_saved is None:
            y_true_saved = y_true
            #print('len(y_true)=', len(y_true))
        else:
            assert np.array_equal(y_true, y_true_saved)
        y_pred = y_pred[:,0]
        y_pred_all.append(y_pred)
    y_pred = np.array(y_pred_all).mean(axis=0)
    return y_true[idx], y_pred[idx] > threshold
    #print('fnr:', (y_pred[y_true == 1] < threshold).sum() / y_true.sum())

# feature distributions from valid sets of given fold
def calc_feature_distrib(feature, fold):
    dir = feature_root / f'{feature}_A{fold}'
    path = dir / 'valid.h5'
    with h5py.File(str(path), 'r') as h5f:
        y_pred = h5f['pred'][()]
        y_true = h5f['true'][()]
    y_true = np.argmax(y_true, axis=-1)
    #y_pred = y_pred[:,1]
    # oversample minority class
    n_major = (y_true == 0).sum()
    n_minor = (y_true == 1).sum()
    assert n_major > n_minor, 'expected minority class is 1'
    y_true_aug = np.array(list(y_true[y_true == 0]) + (list(y_true[y_true == 1]) * int(np.ceil(n_major / n_minor)))[:n_major])
    y_pred_aug = np.array(list(y_pred[y_true == 0]) + (list(y_pred[y_true == 1]) * int(np.ceil(n_major / n_minor)))[:n_major])
    assert (y_true_aug == 0).sum() == (y_true_aug == 1).sum()
    return np.sort(y_pred_aug)
# feature values from test sets
def feature_value(feature, fold, idx, y_true_cm):
    dir = feature_root / f'{feature}_A{fold}'
    path = dir / 'test.h5'
    with h5py.File(str(path), 'r') as h5f:
        y_pred = h5f['pred'][()]
        y_true = h5f['true'][()]
        
    y_true = np.argmax(y_true, axis=-1)
    y_pred = y_pred[:,1]
    assert y_true[idx] == y_true_cm
    return y_pred[idx]

def feature_percentile(feature, name):
    fold, idx = get_fold_idx(name)
    arr = calc_feature_distrib(feature, fold)
    y_true_cm, y_pred_cm = get_true_pred(name)
    value = feature_value(feature, fold, idx, y_true_cm)
    return 100 * (arr < value).sum() / len(arr)

#print(get_fold_idx('ISIC_0015395_downsampled'))
#get_pred('ISIC_0015395_downsampled')
Images_to_test=os.listdir("/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/isic2019/images")
# path_to_images_nipgboard="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/TN_FN"


count_FN=0
count_TN=0

    
if 1:
    for i in Images_to_test:
        l=i.split(".")
        if l[1]=="jpg":
            name=l[0]
            path_to_image_files="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/isic2019/images"
            path_to_images_nipgboard="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/TN_FN"


            if get_true_pred(name)[0]== 1 and get_true_pred(name)[1]==False:
                img=name+".jpg"
                FN_images=os.path.join(path_to_image_files,img)
                modeified_FN_image_name="FN"+"_"+name+"."+"jpg"
                           
                Rename_FN_images=os.path.join(path_to_images_nipgboard,modeified_FN_image_name)
                
                #copy the content
                shutil.copyfile(FN_images,Rename_FN_images)
                
#                 List_original_image.append(img)
#                 List_TN_FN_image.append(modeified_FN_image_name)
#                 List_whole_per.append(feature_percentile("whole", name))
#                 List_colasym_per.append(feature_percentile("colasym", name))
#                 List_center_per.append(feature_percentile("center", name))
#                 List_border_per.append(feature_percentile("border", name))
#                 List_bwv_per.append(feature_percentile("bwv", name))
                count_FN=count_FN+1
                print ("FN :",count_FN)
        
        
                
            elif get_true_pred(name)[0]== 0 and get_true_pred(name)[1]==False:
                img=name+".jpg"
                path_to_image_files="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/isic2019/images"
                path_to_images_nipgboard="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/TN_FN"




                TN_images=os.path.join(path_to_image_files,img)
                modeified_TN_image_name="TN"+"_"+name+"."+"jpg"
                           
                Rename_TN_images=os.path.join(path_to_images_nipgboard,modeified_TN_image_name)
                #copy the content
                shutil.copyfile(TN_images,Rename_TN_images)
                
                count_TN=count_TN+1
                print ("TN :",count_TN)

TN : 1
TN : 2
TN : 3
TN : 4
TN : 5
TN : 6
TN : 7
TN : 8
TN : 9
TN : 10
TN : 11
TN : 12
TN : 13
TN : 14
TN : 15
TN : 16
TN : 17
TN : 18
TN : 19
FN : 1
TN : 20
TN : 21
TN : 22
TN : 23
TN : 24
TN : 25
TN : 26
TN : 27
TN : 28
TN : 29
TN : 30
TN : 31
TN : 32
TN : 33
TN : 34
TN : 35
TN : 36
TN : 37
TN : 38
TN : 39
TN : 40
TN : 41
TN : 42
TN : 43
TN : 44
TN : 45
TN : 46
TN : 47
TN : 48
TN : 49
TN : 50
TN : 51
TN : 52
TN : 53
TN : 54
TN : 55
TN : 56
TN : 57
TN : 58
TN : 59
TN : 60
TN : 61
TN : 62
TN : 63
TN : 64
TN : 65
TN : 66
TN : 67
TN : 68
TN : 69
TN : 70
TN : 71
TN : 72
TN : 73
TN : 74
TN : 75
TN : 76
TN : 77
TN : 78
TN : 79
TN : 80
TN : 81
TN : 82
TN : 83
TN : 84
FN : 2
TN : 85
TN : 86
TN : 87
TN : 88
TN : 89
TN : 90
TN : 91
TN : 92
TN : 93
TN : 94
TN : 95
TN : 96
TN : 97
TN : 98
TN : 99
TN : 100
TN : 101
TN : 102
TN : 103
TN : 104
TN : 105
TN : 106
TN : 107
TN : 108
TN : 109
TN : 110
TN : 111
TN : 112
TN : 113
TN : 114
TN : 115
TN : 116
TN : 117
TN : 118
TN : 119
TN : 120
TN : 121
TN : 

TN : 909
TN : 910
TN : 911
TN : 912
TN : 913
TN : 914
TN : 915
TN : 916
TN : 917
TN : 918
TN : 919
TN : 920
TN : 921
TN : 922
TN : 923
TN : 924
TN : 925
TN : 926
TN : 927
TN : 928
TN : 929
TN : 930
TN : 931
TN : 932
TN : 933
TN : 934
TN : 935
TN : 936
TN : 937
TN : 938
TN : 939
TN : 940
TN : 941
TN : 942
TN : 943
FN : 20
TN : 944
TN : 945
TN : 946
TN : 947
TN : 948
TN : 949
TN : 950
TN : 951
TN : 952
TN : 953
TN : 954
TN : 955
TN : 956
TN : 957
TN : 958
TN : 959
TN : 960
TN : 961
TN : 962
TN : 963
TN : 964
TN : 965
TN : 966
TN : 967
TN : 968
TN : 969
TN : 970
TN : 971
TN : 972
TN : 973
FN : 21
TN : 974
TN : 975
TN : 976
TN : 977
TN : 978
TN : 979
TN : 980
TN : 981
TN : 982
TN : 983
TN : 984
TN : 985
TN : 986
TN : 987
TN : 988
TN : 989
TN : 990
TN : 991
TN : 992
TN : 993
TN : 994
TN : 995
TN : 996
TN : 997
TN : 998
TN : 999
TN : 1000
FN : 22
TN : 1001
TN : 1002
TN : 1003
TN : 1004
TN : 1005
TN : 1006
TN : 1007
TN : 1008
TN : 1009
TN : 1010
TN : 1011
TN : 1012
TN : 1013
TN : 1014
TN : 10

TN : 1723
TN : 1724
TN : 1725
TN : 1726
TN : 1727
TN : 1728
TN : 1729
TN : 1730
TN : 1731
TN : 1732
TN : 1733
TN : 1734
TN : 1735
TN : 1736
TN : 1737
TN : 1738
TN : 1739
TN : 1740
TN : 1741
TN : 1742
TN : 1743
TN : 1744
TN : 1745
TN : 1746
TN : 1747
TN : 1748
TN : 1749
TN : 1750
TN : 1751
TN : 1752
TN : 1753
TN : 1754
TN : 1755
TN : 1756
TN : 1757
TN : 1758
TN : 1759
FN : 39
TN : 1760
TN : 1761
TN : 1762
TN : 1763
TN : 1764
TN : 1765
TN : 1766
TN : 1767
TN : 1768
TN : 1769
TN : 1770
TN : 1771
TN : 1772
TN : 1773
TN : 1774
TN : 1775
TN : 1776
TN : 1777
TN : 1778
TN : 1779
TN : 1780
TN : 1781
TN : 1782
TN : 1783
TN : 1784
TN : 1785
TN : 1786
TN : 1787
TN : 1788
TN : 1789
TN : 1790
TN : 1791
TN : 1792
TN : 1793
TN : 1794
TN : 1795
TN : 1796
TN : 1797
TN : 1798
TN : 1799
TN : 1800
TN : 1801
TN : 1802
TN : 1803
TN : 1804
TN : 1805
TN : 1806
TN : 1807
TN : 1808
TN : 1809
TN : 1810
FN : 40
TN : 1811
TN : 1812
TN : 1813
TN : 1814
TN : 1815
TN : 1816
TN : 1817
TN : 1818
TN : 1819
TN : 1820
TN :

TN : 2530
TN : 2531
FN : 56
TN : 2532
TN : 2533
TN : 2534
FN : 57
TN : 2535
TN : 2536
TN : 2537
TN : 2538
TN : 2539
TN : 2540
TN : 2541
TN : 2542
TN : 2543
TN : 2544
TN : 2545
TN : 2546
TN : 2547
TN : 2548
TN : 2549
TN : 2550
TN : 2551
TN : 2552
TN : 2553
TN : 2554
TN : 2555
TN : 2556
TN : 2557
TN : 2558
TN : 2559
TN : 2560
FN : 58
TN : 2561
TN : 2562
TN : 2563
TN : 2564
TN : 2565
TN : 2566
TN : 2567
TN : 2568
TN : 2569
TN : 2570
TN : 2571
TN : 2572
TN : 2573
TN : 2574
TN : 2575
TN : 2576
TN : 2577
TN : 2578
TN : 2579
TN : 2580
TN : 2581
TN : 2582
TN : 2583
TN : 2584
TN : 2585
TN : 2586
TN : 2587
TN : 2588
TN : 2589
TN : 2590
TN : 2591
TN : 2592
TN : 2593
TN : 2594
TN : 2595
TN : 2596
TN : 2597
TN : 2598
TN : 2599
TN : 2600
TN : 2601
TN : 2602
TN : 2603
TN : 2604
TN : 2605
TN : 2606
TN : 2607
TN : 2608
TN : 2609
TN : 2610
TN : 2611
TN : 2612
TN : 2613
TN : 2614
TN : 2615
TN : 2616
TN : 2617
TN : 2618
TN : 2619
TN : 2620
TN : 2621
TN : 2622
TN : 2623
TN : 2624
TN : 2625
TN : 2626
TN : 2

TN : 3338
TN : 3339
TN : 3340
TN : 3341
TN : 3342
TN : 3343
TN : 3344
TN : 3345
TN : 3346
TN : 3347
TN : 3348
TN : 3349
TN : 3350
TN : 3351
TN : 3352
TN : 3353
TN : 3354
TN : 3355
TN : 3356
TN : 3357
TN : 3358
TN : 3359
TN : 3360
TN : 3361
TN : 3362
TN : 3363
TN : 3364
TN : 3365
TN : 3366
TN : 3367
TN : 3368
TN : 3369
TN : 3370
TN : 3371
TN : 3372
TN : 3373
TN : 3374
TN : 3375
TN : 3376
TN : 3377
TN : 3378
TN : 3379
TN : 3380
TN : 3381
TN : 3382
TN : 3383
TN : 3384
TN : 3385
TN : 3386
TN : 3387
TN : 3388
TN : 3389
TN : 3390
TN : 3391
TN : 3392
TN : 3393
TN : 3394
TN : 3395
TN : 3396
TN : 3397
TN : 3398
TN : 3399
TN : 3400
TN : 3401
TN : 3402
TN : 3403
TN : 3404
TN : 3405
TN : 3406
TN : 3407
TN : 3408
TN : 3409
TN : 3410
TN : 3411
TN : 3412
TN : 3413
TN : 3414
TN : 3415
TN : 3416
TN : 3417
TN : 3418
TN : 3419
TN : 3420
TN : 3421
TN : 3422
TN : 3423
TN : 3424
TN : 3425
TN : 3426
TN : 3427
TN : 3428
TN : 3429
TN : 3430
TN : 3431
TN : 3432
TN : 3433
TN : 3434
TN : 3435
TN : 3436
TN : 3437


...

TN : 12702
TN : 12703
TN : 12704
TN : 12705
TN : 12706
TN : 12707
TN : 12708
TN : 12709
TN : 12710
TN : 12711
TN : 12712
TN : 12713
TN : 12714
TN : 12715
TN : 12716
TN : 12717
TN : 12718
TN : 12719
FN : 318
TN : 12720
TN : 12721
TN : 12722
TN : 12723
TN : 12724
TN : 12725
TN : 12726
TN : 12727
TN : 12728
TN : 12729
TN : 12730
TN : 12731
TN : 12732
TN : 12733
TN : 12734
TN : 12735
TN : 12736
TN : 12737
TN : 12738
TN : 12739
TN : 12740
TN : 12741
TN : 12742
TN : 12743
TN : 12744
TN : 12745
TN : 12746
TN : 12747
TN : 12748
TN : 12749
TN : 12750
TN : 12751
TN : 12752
TN : 12753
TN : 12754
TN : 12755
TN : 12756
TN : 12757
TN : 12758
TN : 12759
TN : 12760
TN : 12761
TN : 12762
TN : 12763
TN : 12764
TN : 12765
TN : 12766
TN : 12767
TN : 12768
TN : 12769
TN : 12770
TN : 12771
TN : 12772
TN : 12773
TN : 12774
FN : 319
TN : 12775
TN : 12776
TN : 12777
TN : 12778
TN : 12779
TN : 12780
TN : 12781
TN : 12782
TN : 12783
TN : 12784
TN : 12785
TN : 12786
TN : 12787
TN : 12788
TN : 12789
TN : 12790
TN 

TN : 13436
TN : 13437
TN : 13438
TN : 13439
TN : 13440
TN : 13441
TN : 13442
TN : 13443
TN : 13444
TN : 13445
TN : 13446
TN : 13447
TN : 13448
TN : 13449
TN : 13450
TN : 13451
TN : 13452
TN : 13453
TN : 13454
TN : 13455
TN : 13456
TN : 13457
TN : 13458
TN : 13459
TN : 13460
TN : 13461
TN : 13462
TN : 13463
TN : 13464
TN : 13465
TN : 13466
TN : 13467
TN : 13468
TN : 13469
TN : 13470
TN : 13471
TN : 13472
TN : 13473
TN : 13474
TN : 13475
TN : 13476
TN : 13477
TN : 13478
TN : 13479
FN : 336
TN : 13480
TN : 13481
TN : 13482
TN : 13483
TN : 13484
TN : 13485
TN : 13486
TN : 13487
TN : 13488
TN : 13489
TN : 13490
TN : 13491
TN : 13492
TN : 13493
TN : 13494
TN : 13495
TN : 13496
TN : 13497
TN : 13498
TN : 13499
TN : 13500
TN : 13501
TN : 13502
TN : 13503
TN : 13504
TN : 13505
TN : 13506
TN : 13507
TN : 13508
TN : 13509
FN : 337
TN : 13510
TN : 13511
TN : 13512
TN : 13513
TN : 13514
TN : 13515
TN : 13516
TN : 13517
TN : 13518
TN : 13519
TN : 13520
TN : 13521
TN : 13522
TN : 13523
TN : 13524
TN 

TN : 14167
TN : 14168
TN : 14169
TN : 14170
TN : 14171
TN : 14172
TN : 14173
TN : 14174
TN : 14175
TN : 14176
TN : 14177
TN : 14178
TN : 14179
TN : 14180
TN : 14181
FN : 353
TN : 14182
TN : 14183
TN : 14184
TN : 14185
TN : 14186
TN : 14187
TN : 14188
TN : 14189
TN : 14190
TN : 14191
TN : 14192
TN : 14193
TN : 14194
TN : 14195
TN : 14196
TN : 14197
TN : 14198
TN : 14199
TN : 14200
TN : 14201
TN : 14202
TN : 14203
TN : 14204
TN : 14205
TN : 14206
TN : 14207
TN : 14208
TN : 14209
TN : 14210
TN : 14211
TN : 14212
TN : 14213
TN : 14214
TN : 14215
TN : 14216
TN : 14217
TN : 14218
TN : 14219
TN : 14220
TN : 14221
TN : 14222
TN : 14223
TN : 14224
TN : 14225
TN : 14226
TN : 14227
TN : 14228
TN : 14229
TN : 14230
TN : 14231
TN : 14232
TN : 14233
TN : 14234
TN : 14235
TN : 14236
TN : 14237
TN : 14238
TN : 14239
TN : 14240
TN : 14241
TN : 14242
TN : 14243
TN : 14244
TN : 14245
TN : 14246
TN : 14247
TN : 14248
TN : 14249
TN : 14250
TN : 14251
TN : 14252
TN : 14253
TN : 14254
TN : 14255
TN : 14256
T

TN : 14898
TN : 14899
TN : 14900
TN : 14901
TN : 14902
TN : 14903
TN : 14904
TN : 14905
TN : 14906
TN : 14907
TN : 14908
TN : 14909
TN : 14910
FN : 372
TN : 14911
TN : 14912
TN : 14913
TN : 14914
TN : 14915
TN : 14916
TN : 14917
TN : 14918
TN : 14919
TN : 14920
TN : 14921
TN : 14922
TN : 14923
TN : 14924
TN : 14925
TN : 14926
TN : 14927
TN : 14928
TN : 14929
TN : 14930
TN : 14931
TN : 14932
TN : 14933
TN : 14934
TN : 14935
TN : 14936
TN : 14937
TN : 14938
TN : 14939
TN : 14940
TN : 14941
TN : 14942
TN : 14943
TN : 14944
TN : 14945
TN : 14946
TN : 14947
TN : 14948
TN : 14949
TN : 14950
TN : 14951
TN : 14952
TN : 14953
TN : 14954
TN : 14955
TN : 14956
TN : 14957
TN : 14958
TN : 14959
TN : 14960
TN : 14961
TN : 14962
TN : 14963
TN : 14964
TN : 14965
TN : 14966
TN : 14967
TN : 14968
TN : 14969
TN : 14970
TN : 14971
TN : 14972
TN : 14973
TN : 14974
TN : 14975
TN : 14976
TN : 14977
TN : 14978
TN : 14979
TN : 14980
TN : 14981
TN : 14982
TN : 14983
TN : 14984
TN : 14985
TN : 14986
TN : 14987
T

TN : 15627
TN : 15628
TN : 15629
TN : 15630
TN : 15631
TN : 15632
TN : 15633
TN : 15634
TN : 15635
TN : 15636
TN : 15637
TN : 15638
TN : 15639
TN : 15640
TN : 15641
TN : 15642
TN : 15643
TN : 15644
TN : 15645
TN : 15646
TN : 15647
TN : 15648
TN : 15649
TN : 15650
TN : 15651
TN : 15652
TN : 15653
TN : 15654
TN : 15655
TN : 15656
TN : 15657
TN : 15658
TN : 15659
FN : 393
TN : 15660
TN : 15661
TN : 15662
TN : 15663
TN : 15664
TN : 15665
TN : 15666
TN : 15667
TN : 15668
TN : 15669
TN : 15670
TN : 15671
TN : 15672
TN : 15673
TN : 15674
TN : 15675
TN : 15676
TN : 15677
TN : 15678
TN : 15679
TN : 15680
TN : 15681
TN : 15682
TN : 15683
TN : 15684
TN : 15685
TN : 15686
TN : 15687
TN : 15688
TN : 15689
TN : 15690
TN : 15691
TN : 15692
TN : 15693
TN : 15694
TN : 15695
TN : 15696
TN : 15697
TN : 15698
TN : 15699
TN : 15700
TN : 15701
TN : 15702
TN : 15703
TN : 15704
TN : 15705
TN : 15706
TN : 15707
TN : 15708
TN : 15709
TN : 15710
TN : 15711
TN : 15712
TN : 15713
TN : 15714
TN : 15715
FN : 394
TN 

In [4]:
print (count_TN)
print (count_FN)

16332
412


In [4]:
for i in (os.listdir("/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/TN_FN")):
    print (i)
    sys.exit()
    


TN_ISIC_0024311.jpg


SystemExit: 

/home/antika/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
len(os.listdir("/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/TN_FN"))

16744

In [3]:
import cv2
import numpy as np
a=cv2.imread("/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/TN_FN/FN_ISIC_0013644_downsampled.jpg")
# b=np.array(a)
# print (b)

# Reduce the original number of TN-FN due to memory issues

folder--> contains the original TN-FN labelled images

new_folder--> contains reduced images of TN-FN labelled images

**Filter all the FN images from oroginal dataset,and 4500 TN images

In [2]:
import os
c=0
c1=0
#os.chdir("/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/images_for_nipgboard")
folder="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/TN_FN"

new_folder="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/kira_final/images"
lis=os.listdir(folder)

for f in lis:

    if "FN" in f:

        shutil.copy(os.path.join(folder, f), new_folder)
        c1=c1+1
        print (c1)

folder="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/TN_FN"

for f in lis:
    
    if "TN" in f:

        shutil.copy(os.path.join(folder, f), new_folder)
        c=c+1
        print(c)
        if c==412:
            break


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [3]:
length=len(os.listdir("/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/kira_final/images"))
length

824

In [6]:
train_len=int(length*0.80)
test_length=int(length-train_len)
test_length

165

In [7]:
import os
c=0
c1=0
#os.chdir("/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/images_for_nipgboard")
folder="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/kira_final/images"
train_folder="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/kira_final/train_img"
test_folder= "/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/kira_final/test_img"
lis=os.listdir(folder)
random.shuffle(lis)
for f in lis:
        if c1<=train_len:
            shutil.copy(os.path.join(folder, f), train_folder)
            c1=c1+1
        else:
            shutil.copy(os.path.join(folder, f), test_folder)
            
        




In [11]:
o=0
for i in (os.listdir("/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/kira_final/test_img")):
          if "TN" in i:
              o=o+1
print (o)        

77


In [33]:
# new_folder="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/kira_eval/random_images"

# import random, os
# l = os.listdir(new_folder)
# random.shuffle(l)

# print (len(l[1100:]))
# print (l[1100:])


In [14]:
# from random import shuffle
# from glob import glob
# files = glob(r"/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/kira_eval/random_images/*jpg")
# shuffle(files)

In [58]:
# folder1="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/random_images"
# new_folder1="/home/antika/nipgboardprojects/nipgboardprojects/nipg-board-v3/skin_antika/kira_eval/images"
# lis=os.listdir(folder1)


# for f in np.random.permutation((len(lis))):
#     print (lis[f])
#     shutil.copy(os.path.join(folder1, lis[f]), new_folder1)
 



# split_data_set


In [ ]:
a=